In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import lightgbm as lgb

In [2]:
data_dir = Path.home() / 'Desktop/kaggle/quora'

[str(p) for p in data_dir.iterdir()]

['/usr/local/google/home/maekawa/Desktop/kaggle/quora/.ipynb_checkpoints',
 '/usr/local/google/home/maekawa/Desktop/kaggle/quora/embeddings.zip',
 '/usr/local/google/home/maekawa/Desktop/kaggle/quora/train.csv.zip',
 '/usr/local/google/home/maekawa/Desktop/kaggle/quora/train_with_length.pickle',
 '/usr/local/google/home/maekawa/Desktop/kaggle/quora/test_with_keywords.pickle',
 '/usr/local/google/home/maekawa/Desktop/kaggle/quora/train_with_keywords.pickle',
 '/usr/local/google/home/maekawa/Desktop/kaggle/quora/sample_submission.csv.zip',
 '/usr/local/google/home/maekawa/Desktop/kaggle/quora/test.csv.zip',
 '/usr/local/google/home/maekawa/Desktop/kaggle/quora/sample_resubmissions.csv',
 '/usr/local/google/home/maekawa/Desktop/kaggle/quora/test_with_length.pickle']

In [7]:
train_with_length = pd.read_pickle(data_dir / 'train_with_length.pickle')
train_with_keywords = pd.read_pickle(data_dir / 'train_with_keywords.pickle')
test_with_length = pd.read_pickle(data_dir / 'test_with_length.pickle')
test_with_keywords = pd.read_pickle(data_dir / 'test_with_keywords.pickle')

In [8]:
train_with_length.head()

,qid,target,question_chars,question_terms
0,00002165364db923c7e6,0,72,13
1,000032939017120e6e44,0,81,16
2,0000412ca6e4628ce2cf,0,67,10
3,000042bf85aa498cd78e,0,57,9
4,0000455dfa3e01eae3af,0,77,15


In [9]:
train_with_keywords.head()

,qid,target,keyword_kill,keyword_killed,keyword_killing,keyword_fuck,keyword_fucking,keyword_penis,keyword_skin,keyword_races,keyword_racism,keyword_racist
0,00002165364db923c7e6,0,False,False,False,False,False,False,False,False,False,False
1,000032939017120e6e44,0,False,False,False,False,False,False,False,False,False,False
2,0000412ca6e4628ce2cf,0,False,False,False,False,False,False,False,False,False,False
3,000042bf85aa498cd78e,0,False,False,False,False,False,False,False,False,False,False
4,0000455dfa3e01eae3af,0,False,False,False,False,False,False,False,False,False,False


In [36]:
train_concat = pd.concat([train_with_length, train_with_keywords.drop(columns=['qid', 'target'])], sort=False, axis=1)
test_concat = pd.concat([test_with_length, test_with_keywords.drop(columns=['qid'])], sort=False, axis=1)

In [37]:
train_concat.head()

,qid,target,question_chars,question_terms,keyword_kill,keyword_killed,keyword_killing,keyword_fuck,keyword_fucking,keyword_penis,keyword_skin,keyword_races,keyword_racism,keyword_racist
0,00002165364db923c7e6,0,72,13,False,False,False,False,False,False,False,False,False,False
1,000032939017120e6e44,0,81,16,False,False,False,False,False,False,False,False,False,False
2,0000412ca6e4628ce2cf,0,67,10,False,False,False,False,False,False,False,False,False,False
3,000042bf85aa498cd78e,0,57,9,False,False,False,False,False,False,False,False,False,False
4,0000455dfa3e01eae3af,0,77,15,False,False,False,False,False,False,False,False,False,False


In [38]:
y = train_concat.target
X = train_concat.drop(columns=['qid', 'target'])

In [39]:
dset = lgb.Dataset(X, y)
params = {}
model = lgb.train(params, dset)

In [40]:
X_test = test_concat.drop(columns=['qid'])

In [42]:
y_test_pred = pd.Series(model.predict(X_test))

In [43]:
y_test_pred.head()

0    0.074785
1    0.032760
2    0.032949
3    0.127251
4    0.035798
dtype: float64

In [48]:
prediction = (y_test_pred > 0.15).map(lambda x: 1 if x else 0)
prediction.sum() / len(y_test_pred)

0.05749512151853823

In [51]:
answer = pd.DataFrame(test_concat.qid)
answer = answer.assign(prediction=prediction)
answer.head()

,qid,prediction
0,00014894849d00ba98a9,0
1,000156468431f09b3cae,0
2,000227734433360e1aae,0
3,0005e06fbe3045bd2a92,0
4,00068a0f7f41f50fc399,0
